In [13]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, Column, String, Float, Integer, Date
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session, relationship
from sqlalchemy import create_engine
from datetime import datetime
from functools import reduce
import requests
from sqlalchemy import func

In [14]:
weather_data_oakville = 'Resources/Oakville_Historical_Weather.csv'
selected_columns = ['HEATING_DEGREE_DAYS', 'MIN_TEMPERATURE','LOCAL_DATE', 'STATION_NAME', 'COOLING_DEGREE_DAYS', 'LOCAL_MONTH', 'LOCAL_DAY', 'LOCAL_YEAR', 'TOTAL_PRECIPITATION', 'SNOW_ON_GROUND', 'MEAN_TEMPERATURE', 'TOTAL_SNOW', 'TOTAL_RAIN', 'MAX_TEMPERATURE', 'ID']
df_weather = pd.read_csv(weather_data_oakville, usecols=selected_columns, encoding="ISO-8859-1", on_bad_lines='skip', low_memory=True, engine='python')
df_weather = df_weather.dropna(subset=['MIN_TEMPERATURE'])
df_weather = df_weather.reset_index(drop=True)
columns_to_fill = ['HEATING_DEGREE_DAYS', 'MIN_TEMPERATURE', 'COOLING_DEGREE_DAYS', 
                   'TOTAL_PRECIPITATION', 'SNOW_ON_GROUND', 'MEAN_TEMPERATURE', 
                   'TOTAL_SNOW', 'TOTAL_RAIN', 'MAX_TEMPERATURE']
df_weather[columns_to_fill] = df_weather[columns_to_fill].fillna(0)
df_weather['LOCAL_DATE'] = pd.to_datetime(df_weather['LOCAL_DATE']).dt.date
df_weather.sample(10)

,HEATING_DEGREE_DAYS,MIN_TEMPERATURE,LOCAL_DATE,STATION_NAME,COOLING_DEGREE_DAYS,LOCAL_MONTH,LOCAL_DAY,LOCAL_YEAR,TOTAL_PRECIPITATION,SNOW_ON_GROUND,MEAN_TEMPERATURE,TOTAL_SNOW,TOTAL_RAIN,MAX_TEMPERATURE,ID
3066,12.0,1.0,1998-10-30,OAKVILLE GERARD,0.0,10,30,1998,0.0,0.0,6.0,0.0,0.0,11.0,6155PD4.1998.10.30
5327,16.2,-0.1,2005-03-20,OAKVILLE GERARD,0.0,3,20,2005,4.2,18.0,1.8,3.0,1.2,3.6,6155PD4.2005.3.20
3550,11.0,2.0,2000-02-26,OAKVILLE GERARD,0.0,2,26,2000,0.0,6.0,7.0,0.0,0.0,12.0,6155PD4.2000.2.26
1877,0.0,19.0,1995-07-21,OAKVILLE GERARD,5.0,7,21,1995,0.0,0.0,23.0,0.0,0.0,27.0,6155PD4.1995.7.21
2434,17.5,-1.5,1997-02-01,OAKVILLE GERARD,0.0,2,1,1997,0.0,16.0,0.5,0.0,0.0,2.5,6155PD4.1997.2.1
3720,0.0,20.5,2000-09-02,OAKVILLE GERARD,3.3,9,2,2000,11.5,0.0,21.3,0.0,11.5,22.0,6155PD4.2000.9.2
889,7.0,5.0,1992-09-25,OAKVILLE GERARD,0.0,9,25,1992,0.0,0.0,11.0,0.0,0.0,17.0,6155PD4.1992.9.25
5887,3.8,9.3,2006-10-01,OAKVILLE GERARD,0.0,10,1,2006,0.0,0.0,14.2,0.0,0.0,19.0,6155PD4.2006.10.1
2033,19.8,-3.5,1995-12-25,OAKVILLE GERARD,0.0,12,25,1995,0.2,7.0,-1.8,0.2,0.0,0.0,6155PD4.1995.12.25
1244,11.5,2.0,1993-10-19,OAKVILLE GERARD,0.0,10,19,1993,0.0,0.0,6.5,0.0,0.0,11.0,6155PD4.1993.10.19


In [15]:
Base = declarative_base()

In [18]:
#Base.metadata.clear()

In [19]:
class WeatherData(Base):
    __tablename__ = 'weather_data'

    id = Column(String, primary_key=True, unique=True) 
    heating_degree_days = Column(Float)
    min_temperature = Column(Float)
    local_date = Column(String)
    station_name = Column(String)
    cooling_degree_days = Column(Float)
    local_month = Column(Integer)
    local_day = Column(Integer)
    local_year = Column(Integer)
    total_precipitation = Column(Float)
    snow_on_ground = Column(Float)
    mean_temperature = Column(Float)
    total_snow = Column(Float)
    total_rain = Column(Float)
    max_temperature = Column(Float)

In [20]:
engine = create_engine('sqlite:///oakville_housing_weather.db')
conn = engine.connect()
#Base.metadata.drop_all(conn)
Base.metadata.create_all(conn)
session = Session(conn)


In [21]:
weather_data_instances = []

for index, row in df_weather.iterrows():
    weather_data_instance = WeatherData(
        id=row['ID'],
        heating_degree_days=row['HEATING_DEGREE_DAYS'],
        min_temperature=row['MIN_TEMPERATURE'],
        local_date=row['LOCAL_DATE'],
        station_name=row['STATION_NAME'],
        cooling_degree_days=row['COOLING_DEGREE_DAYS'],
        local_month=row['LOCAL_MONTH'],
        local_day=row['LOCAL_DAY'],
        local_year=row['LOCAL_YEAR'],
        total_precipitation=row['TOTAL_PRECIPITATION'],
        snow_on_ground=row['SNOW_ON_GROUND'],
        mean_temperature=row['MEAN_TEMPERATURE'],
        total_snow=row['TOTAL_SNOW'],
        total_rain=row['TOTAL_RAIN'],
        max_temperature=row['MAX_TEMPERATURE']
    )
    weather_data_instances.append(weather_data_instance)




In [23]:
session = Session(bind=engine)
session.add_all(weather_data_instances)
session.commit()
weather_count = session.query(func.count(WeatherData.id)).scalar()


print("Number of records in Weather table:", weather_count)

Number of records in Weather table: 5894


In [24]:
session.close()